In [1]:
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
from ultralytics import YOLO

In [3]:
# model = YOLO("yolo11m-pose.pt")
# model.export(format="engine")

In [ ]:
ENGINE_PATH = "../data/models/yolo11m-pose.engine"
VIDEO_INPUT = "../data/3196221-uhd_3840_2160_25fps.mp4"
VIDEO_OUTPUT = "../data/yolo11m_pose_tracked.mp4"
CSV_OUTPUT = "../data/yolo11m_pose_tracked.csv"

In [5]:
POSE_CONNECTIONS = [
    (5, 7), (7, 9),      # left arm
    (6, 8), (8, 10),     # right arm
    (5, 6),              # shoulders
    (5, 11), (6, 12),    # torso
    (11, 13), (13, 15),  # left leg
    (12, 14), (14, 16)   # right leg
]

In [6]:
model = YOLO(ENGINE_PATH)

FileNotFoundError: '../data/models/yolo11m.engine' does not exist

In [ ]:
def draw_pose(frame, kpts, track_id):
    h, w, _ = frame.shape

    for x, y, conf in kpts:
        if conf > 0.3:
            cv2.circle(frame, (int(x), int(y)), 4, (0, 0, 255), -1)

    for a, b in POSE_CONNECTIONS:
        if kpts[a][2] > 0.3 and kpts[b][2] > 0.3:
            x1, y1 = int(kpts[a][0]), int(kpts[a][1])
            x2, y2 = int(kpts[b][0]), int(kpts[b][1])
            cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    if track_id is not None:
        cx, cy = int(kpts[0][0]), int(kpts[0][1])
        cv2.putText(
            frame, f"ID {track_id}",
            (cx, cy - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6, (255, 255, 0), 2
        )


In [ ]:
cap = cv2.VideoCapture(VIDEO_INPUT)

fps = cap.get(cv2.CAP_PROP_FPS)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(VIDEO_OUTPUT, fourcc, fps, (w, h))


In [ ]:
records = []

with tqdm(total=int(cap.get(cv2.CAP_PROP_FRAME_COUNT)), desc="Tracking") as pbar:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        results = model.track(
            frame,
            persist=True,
            conf=0.4,
            iou=0.5,
            tracker="bytetrack.yaml",
            verbose=False
        )

        if results and results[0].keypoints is not None:
            boxes = results[0].boxes
            kpts = results[0].keypoints.xy.cpu().numpy()
            confs = results[0].keypoints.conf.cpu().numpy()

            track_ids = (
                boxes.id.cpu().numpy().astype(int)
                if boxes.id is not None else [None] * len(kpts)
            )

            for i, (kp, kp_conf) in enumerate(zip(kpts, confs)):
                person_id = track_ids[i]
                kpts_full = np.hstack([kp, kp_conf[:, None]])

                draw_pose(frame, kpts_full, person_id)

                row = {
                    "frame": int(cap.get(cv2.CAP_PROP_POS_FRAMES)),
                    "track_id": person_id
                }

                for j, (x, y, c) in enumerate(kpts_full):
                    row[f"kp{j}_x"] = x
                    row[f"kp{j}_y"] = y
                    row[f"kp{j}_conf"] = c

                records.append(row)
        print(type(results[0].keypoints), results[0].keypoints is None)
        out.write(frame)
        pbar.update(1)

cap.release()
out.release()


Tracking:   0%|          | 0/385 [00:00<?, ?it/s]

requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------- ----------- 1.0/1.5 MB 7.2 MB/s eta 0:00:01
   ----------------------------------- ---- 1.3/1.5 MB 2.8 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 3.0 MB/s  0:00:00

requirements: AutoUpdate success  6.0s
WARNING requirements: Restart runtime or rerun command for updates to take effect

Loading yolo11m-pose.engine for TensorRT inference...


Tracking:   0%|          | 1/385 [00:12<1:18:32, 12.27s/it]

<class 'ultralytics.engine.results.Keypoints'> False
<class 'ultralytics.engine.results.Keypoints'> False


Tracking:   1%|          | 3/385 [00:12<18:28,  2.90s/it]  

...

Tracking:  99%|█████████▉| 381/385 [00:57<00:00, 10.09it/s]

<class 'ultralytics.engine.results.Keypoints'> False
<class 'ultralytics.engine.results.Keypoints'> False
<class 'ultralytics.engine.results.Keypoints'> False


Tracking:  99%|█████████▉| 383/385 [00:57<00:00,  9.96it/s]

<class 'ultralytics.engine.results.Keypoints'> False
<class 'ultralytics.engine.results.Keypoints'> False
<class 'ultralytics.engine.results.Keypoints'> False


Tracking: 100%|██████████| 385/385 [00:57<00:00,  6.66it/s]


In [11]:
df = pd.DataFrame(records)
df.to_csv(CSV_OUTPUT, index=False)

print("Done.")
print("Video:", VIDEO_OUTPUT)
print("CSV:", CSV_OUTPUT)


NameError: name 'records' is not defined